In [2]:
import os
import numpy as np

import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
vgg_tag = 'vgg16'
vgg_input_tensor_name = 'image_input:0'
vgg_keep_prob_tensor_name = 'keep_prob:0'
vgg_layer3_out_tensor_name = 'layer3_out:0'
vgg_layer4_out_tensor_name = 'layer4_out:0'
vgg_layer7_out_tensor_name = 'layer7_out:0'

In [ ]:
## Loading TF Model from Path ( .pb, .index and .data files are present  )

with tf.Session() as sess:
    
    vgg_path = os.path.join('/home/ashis/ashis/Udacity/SDC/Term_3/CarND-Semantic-Segmentation/data/', 'vgg')
    print (vgg_path)
    
    meta_buff = tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    
    

    

In [ ]:
## Load directly from .pb file 

#from tf.graph_utils import load_graph

### Importing Pre trained VGG Semantic Segmentatiaon Model

In [ ]:
## Importing a Pre-trained VGG Semantic Segmentatiaon Model

os.chdir('/home/ashis/ashis/Udacity/SDC/Term_3/GPU_TRAINING_Semantic_Segmentation/CarND-Semantic-Segmentation_Model/')

sess = tf.Session()

## Load the Network from .meta (Contains the trained graph)
saver = tf.train.import_meta_graph('vgg_segment-model.meta')

## Load the parameters from .data  ( Contains all the Weights, biases, gradients ..)
saver.restore(sess ,'./vgg_segment-model')


In [ ]:
## Explore Graph Operations

graph = tf.get_default_graph()

# We retrieve the protobuf graph definition
input_graph_def = graph.as_graph_def()


In [ ]:
print (graph.version)

print (len(graph.get_operations()))


In [ ]:
op_names = np.asarray(graph.get_operations())
np.savetxt("vgg_seg-ops.csv", op_names,fmt="%s", delimiter=",")
del op_names

### Freezing Tensorflow Graph 

In [ ]:
from tensorflow.python.framework import graph_util


output_node_names = "Softmax_289" 

# We clear devices to allow TensorFlow to control on which device it will load operations
clear_devices = True

# We use a built-in TF helper to export variables to constants
output_graph_def = graph_util.convert_variables_to_constants(
    sess, # The session is used to retrieve the weights
    input_graph_def, # The graph_def is used to retrieve the nodes 
    output_node_names.split(",") # The output node names are used to select the usefull nodes
) 



In [ ]:
# Finally we serialize and dump the output graph to the filesystem

## freeze graph  for inference 
output_graph = "optimize_model/frozen_model.pb"

with tf.gfile.GFile(output_graph, "wb") as f:
    f.write(output_graph_def.SerializeToString())
print("%d ops in the final graph." % len(output_graph_def.node))

In [ ]:
## Load the frozen Graph

def load_frozen_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
    return graph

In [ ]:
# We use our "load_frozen_graph" function
frozen_graph = load_frozen_graph('optimize_model/frozen_model.pb')

In [ ]:
len(frozen_graph.get_operations())

In [ ]:
# We can verify that we can access the list of operations in the graph
for op in frozen_graph.get_operations():
    print(op.name)

In [ ]:
# We access the input and output nodes 

x = graph.get_tensor_by_name('prefix/image_input')
y = graph.get_tensor_by_name('prefix/Softmax_289')

# We launch a Session
with tf.Session(graph=graph) as sess:
    # Note: we didn't initialize/restore anything, everything is stored in the graph_def
    y_out = sess.run(y, feed_dict={})
    print(y_out) 

### Inference Understanding In Semantic Segmentation

In [2]:
## Importing a Pre-trained VGG Semantic Segmentatiaon Model

os.chdir('/home/ashis/ashis/Udacity/SDC/Term_3/GPU_TRAINING_Semantic_Segmentation/CarND-Semantic-Segmentation_Model/')

sess = tf.Session()

## Load the Network from .meta (Contains the trained graph)
saver = tf.train.import_meta_graph('vgg_segment-model.meta')

## Load the parameters from .data  ( Contains all the Weights, biases, gradients ..)
saver.restore(sess ,'./vgg_segment-model')


In [18]:
### Define the optimize function

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    
    reshaped_logits = tf.reshape(nn_last_layer, (-1, num_classes))
    print ('Layers reshaped_logits ' , reshaped_logits.get_shape)
    
#     reshaped_labels = tf.reshape(correct_label, (-1, num_classes))
#     print ('Layers correct_label  ' , correct_label.get_shape)
#     print ('Layers reshaped_labels ' , reshaped_labels.get_shape)
    
    #cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = reshaped_logits, labels = reshaped_labels))
    
    #train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
    
    return reshaped_logits

In [19]:
num_classes = 2

output_layer = tf.placeholder(dtype = tf.float32, shape = (None, None, None, num_classes)) 

correct_label = tf.placeholder(dtype = tf.float32, shape = (None, None, None, num_classes)) 

learning_rate = tf.placeholder(dtype = tf.float32)

reshaped_logits = optimize(output_layer, correct_label, learning_rate, num_classes)

vgg_input_tensor_name = 'image_input:0'
image_input = tf.get_default_graph().get_tensor_by_name(vgg_input_tensor_name)
print ('===== image_input ',image_input.get_shape)

vgg_keep_prob_tensor_name = 'keep_prob:0'
keep_prob = tf.get_default_graph().get_tensor_by_name(vgg_keep_prob_tensor_name)
print ('===== keep_prob ', keep_prob.get_shape)

sess.run(tf.global_variables_initializer())


Layers reshaped_logits  <bound method Tensor.get_shape of <tf.Tensor 'Reshape_11:0' shape=(?, 2) dtype=float32>>
===== image_input  <bound method Tensor.get_shape of <tf.Tensor 'image_input:0' shape=(?, ?, ?, 3) dtype=float32>>
===== keep_prob  <bound method Tensor.get_shape of <tf.Tensor 'keep_prob:0' shape=<unknown> dtype=float32>>


In [13]:
runs_dir = '/home/ashis/ashis/Udacity/SDC/Term_3/GPU_TRAINING_Semantic_Segmentation/CarND-Semantic-Segmentation_Model/data_inference'
data_dir = '/home/ashis/ashis/Udacity/SDC/Term_3/GPU_TRAINING_Semantic_Segmentation/CarND-Semantic-Segmentation_Model/data_inference'
#keep_prob = 1.0
image_shape = (160, 576)

In [ ]:

for image_file in glob(os.path.join(data_dir , '*.png')):
        
        print ('==== Inside for loop ')
        
        
        image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
        print ('===== IMAGE SHAPE ', image.shape)
        
        print ('===== input_image  SHAPE ', input_image.get_shape)
        print ('===== logits   SHAPE ', logits.get_shape)
        
        #im_softmax = sess.run([tf.nn.softmax(logits)],{keep_prob: 1.0, image_pl: [image]})
    
        im_softmax = sess.run([tf.nn.softmax(logits)], feed_dict={keep_prob: 1.0, input_image: [image]})
        print ('==== im_softmax ', im_softmax.shape)

        im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
        segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
        mask = scipy.misc.toimage(mask, mode="RGBA")
        street_im = scipy.misc.toimage(image)
        street_im.paste(mask, box=None, mask=mask)

        scipy.misc.imsave(runs_dir, street_im)

In [20]:
# import scipy.misc
# from glob import glob

# def get_infernce(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image):
    
#     print ('=========', os.path.join(data_dir, 'image_2', '*.png'))
    
#     for image_file in glob(os.path.join(data_dir , '*.png')):
        
#         print ('==== Inside for loop ')
        
        
#         image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
#         print ('===== IMAGE SHAPE ', image.shape)
        
#         print ('===== input_image  SHAPE ', input_image.get_shape)
#         print ('===== logits   SHAPE ', logits.get_shape)
        
#         #im_softmax = sess.run([tf.nn.softmax(logits)],{keep_prob: 1.0, image_pl: [image]})
    
#         im_softmax = sess.run([tf.nn.softmax(logits)], feed_dict={keep_prob: 1.0, input_image: [image]})
#         print ('==== im_softmax ', im_softmax.shape)

#         im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
#         segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
#         mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
#         mask = scipy.misc.toimage(mask, mode="RGBA")
#         street_im = scipy.misc.toimage(image)
#         street_im.paste(mask, box=None, mask=mask)

#         scipy.misc.imsave(runs_dir, street_im)
    
    

In [21]:
#get_infernce(runs_dir , data_dir ,sess , image_shape,reshaped_logits ,keep_prob , image_input) 


========= /home/ashis/ashis/Udacity/SDC/Term_3/GPU_TRAINING_Semantic_Segmentation/CarND-Semantic-Segmentation_Model/data_inference/image_2/*.png
==== Inside for loop 
===== IMAGE SHAPE  (160, 576, 3)
===== input_image  SHAPE  <bound method Tensor.get_shape of <tf.Tensor 'image_input:0' shape=(?, ?, ?, 3) dtype=float32>>
===== logits   SHAPE  <bound method Tensor.get_shape of <tf.Tensor 'Reshape_11:0' shape=(?, 2) dtype=float32>>


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_11' with dtype float
	 [[Node: Placeholder_11 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_11', defined at:
  File "/home/ashis/anaconda2/envs/tensorflow/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ashis/anaconda2/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ashis/anaconda2/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ashis/anaconda2/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ashis/anaconda2/envs/tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/ashis/anaconda2/